### Each dataset contains different scripts for data ingestion

In [1]:
!pip install datasets

In [2]:
!pip install langchain

In [3]:
import os
import json
import time
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_experimental.text_splitters import SentenceTransformersTokenTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter
from datasets import load_dataset
import pandas as pd
import chromadb
from chromadb.config import Settings
import os
import json
from datasets import load_dataset
import os
import ast
import re
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

In [4]:
# from datasets import load_dataset
# import pandas as pd

# train_stream = load_dataset(
#     "TechQueen24/DoxplainQA",
#     split="train",
#     streaming=True
# )

# df = pd.DataFrame(list(train_stream))
# print(df.shape)
# df.head()


In [5]:
hotpotqa = load_dataset("hotpotqa/hotpot_qa", "fullwiki")

README.md: 0.00B [00:00, ?B/s]

fullwiki/train-00000-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/train-00001-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/validation-00000-of-00001.parqu(…):   0%|          | 0.00/28.0M [00:00<?, ?B/s]

fullwiki/test-00000-of-00001.parquet:   0%|          | 0.00/27.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [6]:
hotpotqa = hotpotqa["train"].to_pandas()
len(hotpotqa)

90447

In [7]:
hotpotqa.iloc[0].context

{'title': array(['Radio City (Indian radio station)',
        'History of Albanian football', 'Echosmith',
        "Women's colleges in the Southern United States",
        'First Arthur County Courthouse and Jail', "Arthur's Magazine",
        '2014–15 Ukrainian Hockey Championship', 'First for Women',
        'Freeway Complex Fire', 'William Rast'], dtype=object),
 'sentences': array([array(["Radio City is India's first private FM radio station and was started on 3 July 2001.",
               ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
               ' It plays Hindi, English and regional songs.',
               ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
               ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that

In [8]:
hotpotqa.to_csv("/data/hotpotqa.csv", index=False)

In [16]:
CHROMA_DB_PATH = "/data/chromadb"
HOTPOT_DATA_PATH = "/data/hotpotqa.csv"
COLLECTION_NAME = "hotpotqa"

CHAR_CHUNK_SIZE = 2000
CHAR_CHUNK_OVERLAP = 200
TOKEN_CHUNK_SIZE = 256
TOKEN_CHUNK_OVERLAP = 32


def _to_py(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (list, tuple)):
        return [_to_py(x) for x in obj]
    if isinstance(obj, dict):
        return {k: _to_py(v) for k, v in obj.items()}
    return obj


def _safe_eval_numpy_repr(s: str):
    if "array(" not in s:
        return None
    safe_globals = {
        "__builtins__": {},
        "array": np.array,
        "nan": np.nan,
        "object": object,
    }
    s2 = s.strip()

    try:
        return eval(s2, safe_globals, {})
    except Exception:
        return None


def process_context(context_obj):

    if isinstance(context_obj, dict):
        ctx = _to_py(context_obj)
        return _context_dict_to_text(ctx)

    if isinstance(context_obj, str):
        s = context_obj.strip()

        try:
            obj = ast.literal_eval(s)
            obj = _to_py(obj)
            if isinstance(obj, dict):
                return _context_dict_to_text(obj)
            return str(obj)
        except Exception:
            pass

        obj = _safe_eval_numpy_repr(s)
        if isinstance(obj, dict):
            obj = _to_py(obj)
            return _context_dict_to_text(obj)

        return s

    return str(context_obj)


def _context_dict_to_text(ctx: dict) -> str:
    titles = ctx.get("title", []) or []
    sentences = ctx.get("sentences", []) or []

    titles = list(titles) if not isinstance(titles, list) else titles
    sentences = list(sentences) if not isinstance(sentences, list) else sentences

    parts = []
    for title, sents in zip(titles, sentences):
        if isinstance(sents, (list, tuple)):
            paragraph = " ".join(str(x).strip() for x in sents if str(x).strip())
        else:
            paragraph = str(sents).strip()

        t = str(title).strip()
        if t and paragraph:
            parts.append(f"{t}: {paragraph}")
        elif paragraph:
            parts.append(paragraph)

    return "\n\n".join(parts).strip()


def chunk_context(context_text: str):
    if not context_text or not context_text.strip():
        return []

    char_chunks = character_splitter.split_text(context_text)
    token_chunks = []
    for c in char_chunks:
        token_chunks.extend(token_splitter.split_text(c))

    return [t.strip() for t in token_chunks if t and t.strip()]


def load_from_dataframe_chunked(df):
    print(f"Processing DataFrame with {len(df)} rows...")

    documents, metadatas, ids = [], [], []

    for row_idx, row in df.iterrows():
        qid = str(row.get("id", row.get("_id", row_idx)))

        context_text = process_context(row["context"])
        chunks = chunk_context(context_text)
        if not chunks:
            continue

        base_meta = {
            "question_id": qid,
            "question": str(row.get("question", "")),
            "answer": str(row.get("answer", "")),
            "type": str(row.get("type", "")),
            "level": str(row.get("level", "")),
        }

        for ci, chunk in enumerate(chunks):
            chunk_id = f"{qid}::c{ci}"
            documents.append(chunk)
            metadatas.append({**base_meta, "chunk_index": ci})
            ids.append(chunk_id)

        if (row_idx + 1) % 100 == 0:
            print(f"  Processed {row_idx + 1}/{len(df)} rows; chunks so far: {len(documents)}")

    return documents, metadatas, ids


def create_chromadb(documents, metadatas, ids, db_path):
    os.makedirs(db_path, exist_ok=True)

    chroma_client = chromadb.PersistentClient(
        path=db_path,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )

    embedding_fn = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

    try:
        chroma_client.delete_collection(COLLECTION_NAME)
    except Exception:
        pass

    collection = chroma_client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_fn,
    )

    print(f"Adding {len(documents)} chunks to Chroma...")
    batch_size = 1000
    for i in range(0, len(documents), batch_size):
        j = min(i + batch_size, len(documents))
        collection.add(documents=documents[i:j], metadatas=metadatas[i:j], ids=ids[i:j])
        print(f"  Added {j}/{len(documents)}")

    print("Done. Collection count:", collection.count())
    return collection


# Splitters
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=CHAR_CHUNK_SIZE,
    chunk_overlap=CHAR_CHUNK_OVERLAP,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
    tokens_per_chunk=TOKEN_CHUNK_SIZE,
)


def main():
    df = pd.read_csv(HOTPOT_DATA_PATH)
    df = df[:10000]
    documents, metadatas, ids = load_from_dataframe_chunked(df)
    create_chromadb(documents, metadatas, ids, CHROMA_DB_PATH)


if __name__ == "__main__":
    main()


Processing DataFrame with 10000 rows...
  Processed 100/10000 rows; chunks so far: 762
  Processed 200/10000 rows; chunks so far: 1475
  Processed 300/10000 rows; chunks so far: 2177
  Processed 400/10000 rows; chunks so far: 2907
  Processed 500/10000 rows; chunks so far: 3598
  Processed 600/10000 rows; chunks so far: 4323
  Processed 700/10000 rows; chunks so far: 5038
  Processed 800/10000 rows; chunks so far: 5726
  Processed 900/10000 rows; chunks so far: 6477
  Processed 1000/10000 rows; chunks so far: 7223
  Processed 1100/10000 rows; chunks so far: 7954
  Processed 1200/10000 rows; chunks so far: 8652
  Processed 1300/10000 rows; chunks so far: 9378
  Processed 1400/10000 rows; chunks so far: 10108
  Processed 1500/10000 rows; chunks so far: 10806
  Processed 1600/10000 rows; chunks so far: 11528
  Processed 1700/10000 rows; chunks so far: 12242
  Processed 1800/10000 rows; chunks so far: 12996
  Processed 1900/10000 rows; chunks so far: 13709
  Processed 2000/10000 rows; chun

In [17]:
ds = load_dataset("yzhuang/narrative_qa")

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [18]:
narative_qa = ds["train"].to_pandas()
len(narative_qa)

32747

In [19]:
narative_qa.to_csv("/data/narative_qa.csv", index=False)

In [20]:
narative_qa.head()

,id,context,question,answer
0,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who is Miss Delmer?,[the elderly spinster aunt of the Earl de Vers...
1,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who does Arabella Mason wed?,"[Ben Keene, Delmar's valet, Ben Keene]"
2,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",How does Percival Keene get his name?,"[Percival is Captain Delmar's first name, and ..."
3,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",Who is the bully that steals Percival's lunch?,"[his teacher, Mr. O'Gallagher, The schoolmaster]"
4,0029bdbe75423337b551e42bb31f9a102785376f,"Percival Keene\n\n At Madeline Hall, an old ma...",How does Percival get even with O'Gallagher af...,[He sets them on fire with the teacher sitting...


In [22]:
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "/data/chromadb")
NARRATIVE_CSV_PATH = os.getenv("NARRATIVE_CSV_PATH", "/data/narative_qa.csv")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "narrativeqa")

CHAR_CHUNK_SIZE = int(os.getenv("CHAR_CHUNK_SIZE", "2000"))
CHAR_CHUNK_OVERLAP = int(os.getenv("CHAR_CHUNK_OVERLAP", "200"))
TOKEN_CHUNK_SIZE = int(os.getenv("TOKEN_CHUNK_SIZE", "256"))
TOKEN_CHUNK_OVERLAP = int(os.getenv("TOKEN_CHUNK_OVERLAP", "32"))

MAX_ROWS = os.getenv("MAX_ROWS", "10000") 
MAX_ROWS = None if str(MAX_ROWS).lower() == "none" else int(MAX_ROWS)

EMBED_MODEL = os.getenv("EMBED_MODEL", "all-MiniLM-L6-v2")


character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=CHAR_CHUNK_SIZE,
    chunk_overlap=CHAR_CHUNK_OVERLAP,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
    tokens_per_chunk=TOKEN_CHUNK_SIZE,
)


def _safe_eval_numpy_array_repr(s: str):
    if not isinstance(s, str):
        return None
    if "array(" not in s:
        return None

    safe_globals = {
        "__builtins__": {},
        "array": np.array,
        "nan": np.nan,
        "object": object,
    }
    try:
        return eval(s.strip(), safe_globals, {})
    except Exception:
        return None

def parse_answer_first(answer_obj) -> str:
    answers = []

    if isinstance(answer_obj, np.ndarray):
        answers = answer_obj.tolist()
    elif isinstance(answer_obj, (list, tuple)):
        answers = list(answer_obj)
    elif isinstance(answer_obj, str):
        s = answer_obj.strip()
        try:
            obj = ast.literal_eval(s)
            if isinstance(obj, (list, tuple)):
                answers = list(obj)
            else:
                answers = [obj]
        except Exception:
            arr = _safe_eval_numpy_array_repr(s)
            if isinstance(arr, np.ndarray):
                answers = arr.tolist()
            else:
                answers = [s] if s else []
    else:
        answers = [answer_obj]

    answers = [str(a).strip() for a in answers if a is not None and str(a).strip()]
    return answers[0] if answers else ""

def clean_context(context: str) -> str:
    return "" if context is None else str(context).strip()

def chunk_text(text: str):
    if not text or not text.strip():
        return []
    char_chunks = character_splitter.split_text(text)
    token_chunks = []
    for c in char_chunks:
        token_chunks.extend(token_splitter.split_text(c))
    return [t.strip() for t in token_chunks if t and t.strip()]


def build_records(df: pd.DataFrame):
    documents, metadatas, ids = [], [], []

    for row_idx, row in df.iterrows():
        story_id = str(row.get("id", row.get("_id", "unknown"))).strip()

        question_id = f"{story_id}::q{row_idx}"

        question = str(row.get("question", "")).strip()
        context_text = clean_context(row.get("context", ""))

        chunks = chunk_text(context_text)
        if not chunks:
            continue

        answer = parse_answer_first(row.get("answer", ""))

        base_meta = {
            "question_id": question_id,   
            "story_id": story_id,        
            "question": question,
            "answer": answer,
            "dataset": "narrativeqa",
        }

        for ci, chunk in enumerate(chunks):
            chunk_id = f"{question_id}::c{ci}"
            documents.append(chunk)
            metadatas.append({**base_meta, "chunk_index": int(ci)})
            ids.append(chunk_id)

        if (row_idx + 1) % 100 == 0:
            print(f"  Processed {row_idx + 1}/{len(df)} rows; chunks so far: {len(documents)}")

    return documents, metadatas, ids


def write_chroma(documents, metadatas, ids):
    os.makedirs(CHROMA_DB_PATH, exist_ok=True)

    client = chromadb.PersistentClient(
        path=CHROMA_DB_PATH,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )

    embedding_fn = SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

    try:
        client.delete_collection(COLLECTION_NAME)
    except Exception:
        pass

    col = client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_fn,
    )

    print(f"Adding {len(documents)} chunks to Chroma collection '{COLLECTION_NAME}' ...")
    batch_size = 1000
    for i in range(0, len(documents), batch_size):
        j = min(i + batch_size, len(documents))
        col.add(documents=documents[i:j], metadatas=metadatas[i:j], ids=ids[i:j])
        print(f"  Added {j}/{len(documents)}")

    print("Done. Collection count:", col.count())
    return col

def main():
    df = pd.read_csv(NARRATIVE_CSV_PATH)

    df = df[:10000]
    if MAX_ROWS is not None:
        df = df.iloc[:MAX_ROWS].copy()

    print(f"Loaded NarrativeQA rows: {len(df)} from {NARRATIVE_CSV_PATH}")
    docs, metas, ids = build_records(df)
    write_chroma(docs, metas, ids)

if __name__ == "__main__":
    main()


Loaded NarrativeQA rows: 10000 from /data/narative_qa.csv
  Processed 100/10000 rows; chunks so far: 53801
  Processed 200/10000 rows; chunks so far: 83063
  Processed 300/10000 rows; chunks so far: 132951
  Processed 400/10000 rows; chunks so far: 181660
  Processed 500/10000 rows; chunks so far: 206084
  Processed 600/10000 rows; chunks so far: 243811
  Processed 700/10000 rows; chunks so far: 272014
  Processed 800/10000 rows; chunks so far: 299544
  Processed 900/10000 rows; chunks so far: 410159
  Processed 1000/10000 rows; chunks so far: 471844
  Processed 1100/10000 rows; chunks so far: 507953
  Processed 1200/10000 rows; chunks so far: 538601
  Processed 1300/10000 rows; chunks so far: 565662
  Processed 1400/10000 rows; chunks so far: 599167
  Processed 1500/10000 rows; chunks so far: 629709
  Processed 1600/10000 rows; chunks so far: 656191
  Processed 1700/10000 rows; chunks so far: 685803
  Processed 1800/10000 rows; chunks so far: 754682
  Processed 1900/10000 rows; chunks

In [31]:
ds = load_dataset("qiaojin/PubMedQA", "pqa_unlabeled")

pqa_unlabeled/train-00000-of-00001.parqu(…):   0%|          | 0.00/66.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61249 [00:00<?, ? examples/s]

In [32]:
pubmedqa = ds["train"].to_pandas()
len(pubmedqa)

61249

In [33]:
pubmedqa

,pubid,question,context,long_answer
0,14499029,Is naturopathy as effective as conventional th...,{'contexts': ['Although the use of alternative...,Naturopathy appears to be an effective alterna...
1,14499049,Can randomised trials rely on existing electro...,"{'contexts': ['To estimate the feasibility, ut...",Routine data have the potential to support hea...
2,14499672,Is laparoscopic radical prostatectomy better t...,{'contexts': ['To compare morbidity in two gro...,The results of our non-randomized study show t...
3,14499773,Does bacterial gastroenteritis predispose peop...,{'contexts': ['Irritable bowel syndrome (IBS) ...,Symptoms consistent with IBS and functional di...
4,14499777,Is early colonoscopy after admission for acute...,{'contexts': ['Urgent colonoscopy has been pro...,No significant association is apparent between...
...,...,...,...,...
61244,10632750,Diversion colitis in children: an iatrogenic a...,{'contexts': ['Diversion colitis (DC) is a loc...,Histological features of DC in children are ve...
61245,10632796,Raising research awareness among midwives and ...,{'contexts': ['The primary aim of the study wa...,The introduction of clinical governance challe...
61246,10632828,Delivery of primary care to women. Do women's ...,{'contexts': ['Women's health centers have bee...,"These results suggest that, at least in this s..."
61247,10633786,Analysis of failures after whole abdominal irr...,{'contexts': ['To evaluate failures and to inv...,General prophylactic enclosure of the inguinal...


In [34]:
pubmedqa.to_csv("/data/pubmedqa.csv", index=False)

In [35]:
pubmedqa.iloc[0].context

{'contexts': array(['Although the use of alternative medicine in the United States is increasing, no published studies have documented the effectiveness of naturopathy for treatment of menopausal symptoms compared to women receiving conventional therapy in the clinical setting.',
        'To compare naturopathic therapy with conventional medical therapy for treatment of selected menopausal symptoms.',
        'A retrospective cohort study, using abstracted data from medical charts.',
        'One natural medicine and six conventional medical clinics at Community Health Centers of King County, Washington, from November 1, 1996, through July 31, 1998.',
        'Women aged 40 years of age or more with a diagnosis of menopausal symptoms documented by a naturopathic or conventional physician.',
        'Improvement in selected menopausal symptoms.',
        'In univariate analyses, patients treated with naturopathy for menopausal symptoms reported higher monthly incomes ($1848.00 versus $8

In [42]:
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "/data/chromadb")
PUBMEDQA_CSV_PATH = os.getenv("PUBMEDQA_CSV_PATH", "/data/pubmedqa.csv")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "pubmedqa")

CHAR_CHUNK_SIZE = int(os.getenv("CHAR_CHUNK_SIZE", "2000"))
CHAR_CHUNK_OVERLAP = int(os.getenv("CHAR_CHUNK_OVERLAP", "200"))
TOKEN_CHUNK_SIZE = int(os.getenv("TOKEN_CHUNK_SIZE", "256"))
TOKEN_CHUNK_OVERLAP = int(os.getenv("TOKEN_CHUNK_OVERLAP", "32"))

MAX_ROWS = os.getenv("MAX_ROWS", "10000")  # "None" or integer
MAX_ROWS = None if str(MAX_ROWS).lower() == "none" else int(MAX_ROWS)

EMBED_MODEL = os.getenv("EMBED_MODEL", "all-MiniLM-L6-v2")

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=CHAR_CHUNK_SIZE,
    chunk_overlap=CHAR_CHUNK_OVERLAP,
)

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=TOKEN_CHUNK_OVERLAP,
    tokens_per_chunk=TOKEN_CHUNK_SIZE,
)

def _safe_eval_numpy_array_repr(s: str):
    if not isinstance(s, str):
        return None
    if "array(" not in s:
        return None

    safe_globals = {
        "__builtins__": {},
        "array": np.array,
        "nan": np.nan,
        "object": object,
    }
    try:
        return eval(s.strip(), safe_globals, {})
    except Exception:
        return None


def _coerce_to_list(x):
    if x is None:
        return []
    if isinstance(x, np.ndarray):
        return x.tolist()
    if isinstance(x, (list, tuple)):
        return list(x)
    return [x]


def parse_answer_long(answer_obj) -> str:
    if answer_obj is None:
        return ""
    if isinstance(answer_obj, str):
        return answer_obj.strip()

    vals = _coerce_to_list(answer_obj)
    vals = [str(v).strip() for v in vals if v is not None and str(v).strip()]
    return vals[0] if vals else ""


def parse_contexts_from_context_field(context_field):
    if context_field is None:
        return []

    if isinstance(context_field, dict):
        contexts = context_field.get("contexts", [])
        contexts = _coerce_to_list(contexts)
        return [str(c).strip() for c in contexts if c is not None and str(c).strip()]

    s = str(context_field).strip()
    if not s:
        return []

    if (s.startswith("{") and s.endswith("}")) or (s.startswith("[") and s.endswith("]")):
        obj = None
        try:
            obj = ast.literal_eval(s)
        except Exception:
            obj = None

        if obj is None:
            try:
                obj = json.loads(s)
            except Exception:
                obj = None

        if isinstance(obj, dict):
            ctx = obj.get("contexts", [])
            ctx = _coerce_to_list(ctx)

            if len(ctx) == 1 and isinstance(ctx[0], str) and "array(" in ctx[0]:
                arr = _safe_eval_numpy_array_repr(ctx[0])
                if isinstance(arr, np.ndarray):
                    ctx = arr.tolist()

            return [str(c).strip() for c in ctx if c is not None and str(c).strip()]

    try:
        obj = ast.literal_eval(s)
        if isinstance(obj, dict):
            ctx = obj.get("contexts", [])
            ctx = _coerce_to_list(ctx)
            return [str(c).strip() for c in ctx if c is not None and str(c).strip()]
    except Exception:
        pass
    if "contexts" in s and "array(" in s:
        idx = s.find("contexts")
        arr_idx = s.find("array(", idx)
        if arr_idx != -1:
            sub = s[arr_idx:]
            cut = sub.find("),")
            if cut != -1:
                sub = sub[: cut + 1]
            arr = _safe_eval_numpy_array_repr(sub)
            if isinstance(arr, np.ndarray):
                ctx = arr.tolist()
                return [str(c).strip() for c in ctx if c is not None and str(c).strip()]

    return []


def chunk_text(text: str):
    if not text or not str(text).strip():
        return []
    text = str(text).strip()
    char_chunks = character_splitter.split_text(text)
    token_chunks = []
    for c in char_chunks:
        token_chunks.extend(token_splitter.split_text(c))
    return [t.strip() for t in token_chunks if t and t.strip()]


def make_context_text(contexts):

    contexts = [str(c).strip() for c in (contexts or []) if c is not None and str(c).strip()]
    if not contexts:
        return ""
    return "\n".join(contexts)

def build_records(df: pd.DataFrame):
    documents, metadatas, ids = [], [], []

    for row_idx, row in df.iterrows():
        pubid = str(row.get("pubid", row.get("pub_id", row.get("id", "unknown")))).strip()

        question_id = f"{pubid}::q{row_idx}"

        question = str(row.get("question", "")).strip()

        contexts = parse_contexts_from_context_field(row.get("context", ""))
        context_text = make_context_text(contexts)

        chunks = chunk_text(context_text)
        if not chunks:
            continue

        long_answer = parse_answer_long(row.get("long_answer", row.get("answer", "")))

        base_meta = {
            "question_id": question_id, 
            "pubid": pubid,
            "question": question,
            "answer": long_answer,
            "dataset": "pubmedqa",
        }

        for ci, chunk in enumerate(chunks):
            chunk_id = f"{question_id}::c{ci}"
            documents.append(chunk)
            metadatas.append({**base_meta, "chunk_index": int(ci)})
            ids.append(chunk_id)

        if (row_idx + 1) % 1000 == 0:
            print(f"  Processed {row_idx + 1}/{len(df)} rows; chunks so far: {len(documents)}")

    return documents, metadatas, ids

def write_chroma(documents, metadatas, ids):
    os.makedirs(CHROMA_DB_PATH, exist_ok=True)

    client = chromadb.PersistentClient(
        path=CHROMA_DB_PATH,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )

    embedding_fn = SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

    try:
        client.delete_collection(COLLECTION_NAME)
    except Exception:
        pass

    col = client.get_or_create_collection(
        name=COLLECTION_NAME,
        embedding_function=embedding_fn,
    )

    print(f"Adding {len(documents)} chunks to Chroma collection '{COLLECTION_NAME}' ...")
    batch_size = 1000
    for i in range(0, len(documents), batch_size):
        j = min(i + batch_size, len(documents))
        col.add(documents=documents[i:j], metadatas=metadatas[i:j], ids=ids[i:j])
        print(f"  Added {j}/{len(documents)}")

    print("Done. Collection count:", col.count())
    return col


def main():
    df = pd.read_csv(PUBMEDQA_CSV_PATH)

    if MAX_ROWS is not None:
        df = df.iloc[:MAX_ROWS].copy()

    print(f"Loaded PubMedQA rows: {len(df)} from {PUBMEDQA_CSV_PATH}")
    docs, metas, ids = build_records(df)
    write_chroma(docs, metas, ids)


if __name__ == "__main__":
    main()


Loaded PubMedQA rows: 10000 from /data/pubmedqa.csv
  Processed 1000/10000 rows; chunks so far: 885
  Processed 3000/10000 rows; chunks so far: 2652
  Processed 4000/10000 rows; chunks so far: 3494
  Processed 5000/10000 rows; chunks so far: 4394
  Processed 6000/10000 rows; chunks so far: 5280
  Processed 7000/10000 rows; chunks so far: 6173
  Processed 9000/10000 rows; chunks so far: 7963
  Processed 10000/10000 rows; chunks so far: 8836
Adding 8836 chunks to Chroma collection 'pubmedqa' ...
  Added 1000/8836
  Added 2000/8836
  Added 3000/8836
  Added 4000/8836
  Added 5000/8836
  Added 6000/8836
  Added 7000/8836
  Added 8000/8836
  Added 8836/8836
Done. Collection count: 8836
